In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate

from langchain import LLMChain
from langchain.memory import ConversationBufferMemory

from langchain.memory import ChatMessageHistory
from langchain_core.messages import HumanMessage, AIMessage

os.environ["OPENAI_API_KEY"] = ""

In [ ]:
def mail_generation(model, context):
    # Function to generate a promotional mail based on a given context using a language model
    prompt = ChatPromptTemplate.from_messages([("system", "You are a telecommunication company agent and your job is to prevent customers from \
                                                churning by generating promotional mails without allowing them to know you know they have plans \
                                                of leaving You may be asked to generate a mail given a context." ),
                                               ("user", "{mail_description}")],)
    
    output = model.invoke(prompt.format(mail_description=context))
    return output

def mail_revamp(model, mail, suggestion):
    # Function to suggest corrections and revamp a generated mail
    prompt = PromptTemplate(input_variables= ["mail", "suggestion"], 
                            template = "You are a telecommunication company agent and your job is to revamp the promotional mails to be sent to the customers. \
                            Revamp this mail {mail} based on these suggested {suggestion} correction only. Don't go beyond the corrections requested")
    
    output = model.invoke(prompt.format(mail = mail, suggestion = suggestion))
    return output

In [ ]:
text = "You are a telecommunication company agent and your job is to prevent customers from churning by generating promotional\
      mails without allowing them to know you know they have plans of leaving. You may be asked to generate a mail given a context,\
         you can also be asked to revamp a given mail. In any case you are required to stay in your limit do not give suggestion except prompted to" 


llm = ChatOpenAI()
context = "Generate a mail for a customer that is likely to leave the company's product. We have some good network upgrade coming to his/her location that may help change his mind"

output = mail_generation(llm, text, context)
print(output.content)

In [ ]:
template = """You are a telecommunication company chatbot and your job is to generating promotional mails. \
        Your goal is to converse with an agent and list suject ideas for promotional mail.
        If an option is selected, generate a promotional mail for the customer based on the suggestion from our agent, \
        ask for corrections and revamp the mail solely on this corrections if any.

        {chat_history}
        Human: {human_input}
        """

prompt = PromptTemplate(input_variables=["chat_history", "human_input"],
                        template=template)
memory = ConversationBufferMemory(memory_key="chat_history")
llm_chain = LLMChain(
                    llm = ChatOpenAI(),
                    prompt = prompt,
                    memory = memory,
                    verbose = False
                )

In [ ]:
llm_chain.invoke("Hello")

out = llm_chain.invoke("I need ideas on mail i can generate for a very good customer showing signs of leaving")
print(out['chat_history'])

out = llm_chain.invoke("Yes! Generate a mail on the fourth suggestion")
print(out['chat_history'])

In [ ]:
template = """You are a chatbot and your job is to generate promotional mail subject ideas for the company while \
                having a chat for the agent. Ask if the idea is fine and ask which option is fine after all conversation"""

history = [HumanMessage(content = "Hello"), 
           AIMessage(content="Hello! How may i help you?")]


def mail_revamp(model, context, history):
    # Function to suggest corrections and revamp a generated mail
    prompt = ChatPromptTemplate.from_messages([("system", template), 
                                               ("placeholder", "{history}"), 
                                               ("user", "{input_text}")])
    
    output = model.invoke(prompt.format(history = history, input_text= context))
    return output

llm = ChatOpenAI()
satisfied = 0
while not satisfied:

    context = "Promotional mail ideas please?"
    history.append(HumanMessage(content=context))
    output = mail_revamp(llm, context, history)
    
print(output.content)